In [365]:
%matplotlib inline
import pandas as pd
from datetime import datetime
import quandl
import numpy as np
import matplotlib.pyplot as plt
import ssl
import sklearn.preprocessing as prep
import os
import datetime
import fix_yahoo_finance as yf
from pandas_datareader import data as pdr
yf.pdr_override()

In [34]:
quandl.ApiConfig.api_key = "xoemPs9s3Dtwf3aQxQ-3"
start="2010-01-02"

INTC = pdr.get_data_yahoo("INTC",start=start)
MSFT = pdr.get_data_yahoo("MSFT",start=start)
AMZN = pdr.get_data_yahoo("AMZN",start=start)
NVDA = pdr.get_data_yahoo("NVDA",start=start)
GOOG = pdr.get_data_yahoo("GOOG",start=start)

# macro data 
EUROSTOXX50 = pdr.get_data_yahoo("FEZ",start=start)
SNP500 = pdr.get_data_yahoo("^GSPC",start=start)
CONS_DESCRETE = pdr.get_data_yahoo("XLY",start=start)
SNP_TECH = pdr.get_data_yahoo("XLK",start=start)
NASDAQ_COMP = pdr.get_data_yahoo("^IXIC",start=start)



In [354]:
# FRED & Treasury
yieldr = quandl.get("USTREASURY/REALYIELD", start_date=start, verify=False)
consumer_price_index = quandl.get("FRED/CPIAUCSL", start_date=start, verify=False)
gdpc = quandl.get("FRED/GDPC1", start_date=start, verify=False)
tnote_y10 = quandl.get("FRED/DGS10", start_date=start, verify=False)
financial_stress = quandl.get("FRED/STLFSI", start_date=start, verify=False)
effective_federal_fund_rate = quandl.get("FRED/FEDFUNDS", start_date=start, verify=False)


In [405]:
result = pd.concat([INTC, MSFT[['Close']].rename(columns={'Close': 'MSFT'})], axis=1)
result = pd.concat([result, AMZN[['Close']].rename(columns={'Close': 'AMZN'})], axis=1)
result = pd.concat([result, NVDA[['Close']].rename(columns={'Close': 'NVDA'})], axis=1)
result = pd.concat([result, GOOG[['Close']].rename(columns={'Close': 'GOOG'})], axis=1)
result = pd.concat([result, EUROSTOXX50[['Close']].rename(columns={'Close': 'EUROSTOXX50'})], axis=1)
result = pd.concat([result, SNP500[['Close']].rename(columns={'Close': 'SNP500'})], axis=1)
result = pd.concat([result, CONS_DESCRETE[['Close']].rename(columns={'Close': 'CONS_DESCRETE'})], axis=1)
result = pd.concat([result, SNP_TECH[['Close']].rename(columns={'Close': 'SNP_TECH'})], axis=1)
result = pd.concat([result, NASDAQ_COMP[['Close']].rename(columns={'Close': 'NASDAQ_COMP'})], axis=1)
# fred
result = pd.concat([result, yieldr[['5 YR']].rename(columns={'5 YR': 'yield_5y'})], axis=1)
result = pd.concat([result, tnote_y10.rename(columns={'Value': 'Tnote_10y'})], axis=1)

result = result.drop('Adj Close', axis=1)
result['Weekday'] = result.index.weekday
result['Daymonth'] = result.index.day
result['Month'] = result.index.month
result['Weakyear'] = result.index.weekofyear

result = result.dropna()

In [406]:
result[['yield_5y']] = result[['yield_5y']].replace(0, 0.0000001)
result[['yield_5y']] = result[['yield_5y']]+10

In [407]:
def logReturns(df):
    log_ret = np.log(df) - np.log(df.shift(1))
    return log_ret

In [408]:
logRet = result[['MSFT','AMZN','NVDA','GOOG','EUROSTOXX50','SNP500','CONS_DESCRETE','SNP_TECH','NASDAQ_COMP','yield_5y','Tnote_10y']].apply(logReturns)

In [409]:
logRet.columns = [str(col) + '_logReturns' for col in logRet.columns]

In [410]:
result = pd.concat([result, logRet], axis=1)
scaler = prep.MaxAbsScaler().fit(result[['Volume']].values)
result['Volume_Scaled'] = scaler.transform(result[['Volume']].values) 

In [411]:
result.head(3)

,Open,High,Low,Close,Volume,MSFT,AMZN,NVDA,GOOG,EUROSTOXX50,...,NVDA_logReturns,GOOG_logReturns,EUROSTOXX50_logReturns,SNP500_logReturns,CONS_DESCRETE_logReturns,SNP_TECH_logReturns,NASDAQ_COMP_logReturns,yield_5y_logReturns,Tnote_10y_logReturns,Volume_Scaled
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-04,20.790001,21.030001,20.73,20.879999,47800900.0,30.950001,133.899994,18.490000,311.349976,42.900002,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.240202
2010-01-05,20.940001,20.990000,20.60,20.870001,52357700.0,30.959999,134.690002,18.760000,309.978882,42.700001,...,0.014497,-0.004413,-0.004673,0.003111,0.003660,-0.001290,0.000126,-0.007634,-0.020998,0.263101
2010-01-06,20.809999,20.930000,20.67,20.799999,40037400.0,30.770000,132.250000,18.879999,302.164703,42.639999,...,0.006376,-0.025532,-0.001406,0.000545,0.001328,-0.011251,-0.003306,0.000000,0.020998,0.201190


In [434]:
momentum = result['Close'].diff(periods=5)
momentum = momentum[~np.isnan(momentum)]
scaler1 = prep.MinMaxScaler(copy=True, feature_range=(-1, 1)).fit(momentum)


ValueError: Expected 2D array, got 1D array instead:
array=[ 0.070002 -0.26      0.16     ...,  1.959999  1.73      1.599998].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.